In [ ]:
!pip install transformers
!pip install torch
!pip install newspaper3k

In [ ]:
import nltk
nltk.download('punkt')
from textblob import TextBlob
from newspaper import Article

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [3]:
from dataclasses import dataclass

In [4]:
@dataclass
class Mood:
    emoji: str
    sentiment: float

In [ ]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = AutoModel.from_pretrained("dbmdz/bert-base-turkish-cased")


In [6]:
def sentiment_analysis(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    labels = ['negative', 'neutral', 'positive']
    result = {labels[i]: scores[i] for i in range(len(labels))}
    return result


In [16]:
def get_mood(input_text: str, *, threshold: float) -> Mood:
    sentiment: float = TextBlob(input_text).sentiment.polarity


    friendly_treshold: float = threshold
    hostile_treshold: float = -threshold

    if sentiment > friendly_treshold:
        return Mood("😊", sentiment)
    elif sentiment < hostile_treshold:
        return Mood("😠", sentiment)
    else:
        return Mood("😐", sentiment)



url = "https://tr.euronews.com/2023/08/16/avrupali-arastirmacilar-buzadam-otzi-anadoludan-geldi-koyu-tenli-ve-keldi"
article = Article(url)
article.download()
article.parse()
article.nlp()

text = article.summary
print(text)

blob = TextBlob(text)
sentiment = blob.sentiment.polarity # 0 to 1
print(sentiment)

mood = get_mood(text, threshold=0.1)
print(mood)

By euronewsMevcut bilgilere göre boyu 1.60 metrenin biraz altında olan Ötzi, yaklaşık 50 kg ağırlığındaydı ve bir okla öldürülmüştü.
REKLAMAlmanya'da Max Planck Antropoloji Enstitüsü'nün yaptığı araştırmaya göre Avrupa'nın en ünlü mumyası buzadam Ötzi'nin ataları Anadolu'dan geldi.
Araştırma sonuçlarına göre beş bin yaşından büyük olan Ötzi sanılanın aksine daha koyu bir tene sahipti.
Araştırmacılara göre Ötzi, Avrupalı çağdaşlarıyla karşılaştırıldığında, Anadolu'dan göç eden ilk çiftçilerle alışılmadık derecede yüksek bir genetik yüzdeye sahip.
Mevcut bilgilere göre boyu 1.60 metrenin biraz altında olan Ötzi, yaklaşık 50 kg ağırlığındaydı ve bir okla öldürülmüştü.
0.0
Mood(emoji='😐', sentiment=0.0)
